In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
block_size = 64
batches = 32
train_loop = 5000
learning_rate = 5e-4
train_split = 0.8
debug_loop = 500
embeds = 512
heads = 4
layers = 8
dropout = 0.2

cuda


In [2]:
chars = ""
with open('grammar.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '}', '´', 'Á', 'à', 'á', 'æ', 'è', 'é', 'í', 'ò', 'ó', 'ù', 'ú', 'Ā', 'ā', 'Ă', 'ă', 'ē', 'ĕ', 'ħ', 'Ī', 'ī', 'Ĭ', 'ĭ', 'ō', 'Ŏ', 'ŏ', 'œ', 'Ū', 'ū', 'ŭ', 'ȳ', '̄', '̆', 'Α', 'Κ', 'Λ', 'Ν', 'Ο', 'Τ', '‘', '’', '“', '”', '⪥', '\ufeff']
126


In [3]:
str2int = { ch:i for i,ch in enumerate(chars) }
int2str = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [ str2int[c] for c in s ]
decode = lambda l: ''.join([ int2str[i] for i in l ])
data = torch.tensor(encode(text), dtype=torch.long).to(device)
print(data[:100])

tensor([125,   0,   0,   0,   0,  39,  71,  68,  57,  74,  56,  58,  57,   1,
         55,  78,   1,  26,  61,  71,  62,  72,   1,  26,  74,  71,  67,  68,
         76,   1,  54,  67,  57,   1,  73,  61,  58,   1,  38,  67,  65,  62,
         67,  58,   1,  27,  62,  72,  73,  71,  62,  55,  74,  73,  58,  57,
          0,  39,  71,  68,  68,  59,  71,  58,  54,  57,  62,  67,  60,   1,
         43,  58,  54,  66,   1,  54,  73,   1,  61,  73,  73,  69,  21,  10,
         10,  76,  76,  76,   9,  69,  60,  57,  69,   9,  67,  58,  73,   1,
          4,  43], device='cuda:0')


In [4]:
n = int(train_split*len(data))
train = data[:n]
valid = data[n:]

def get_batch(split):
    data = train if split=='train' else valid
    ix = torch.randint(len(data) - block_size, (batches,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[ 73,  61,  58,  ...,  71,  75,  58],
        [ 66,  74,  56,  ...,  58,   1, 122],
        [  0,  62,  67,  ...,  68,  74,  73],
        ...,
        [ 73,  68,   1,  ...,   1,  67,  62],
        [ 58,   7,   1,  ...,  68,  76,  58],
        [ 68,  59,   1,  ...,  25,  71,  68]], device='cuda:0')
target:
tensor([[ 61,  58,   0,  ...,  75,  58,   1],
        [ 74,  56,  61,  ...,   1, 122,  69],
        [ 62,  67,  59,  ...,  74,  73,  52],
        ...,
        [ 68,   1,  73,  ...,  67,  62,  56],
        [  7,   1,  69,  ...,  76,  58,  75],
        [ 59,   1,  73,  ...,  71,  68,  76]], device='cuda:0')


In [5]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(embeds, head_size, bias=False)
        self.query = nn.Linear(embeds, head_size, bias=False)
        self.value = nn.Linear(embeds, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-.5
        wei = wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, embeds)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(embeds, 4*embeds),
            nn.ReLU(),
            nn.Linear(4*embeds, embeds),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, embeds, heads):
        super().__init__()
        head_size = embeds // heads
        self.sa = MultiHeadAttention(heads, head_size)
        self.ffwd = FeedForward(embeds)
        self.ln1 = nn.LayerNorm(embeds)
        self.ln2 = nn.LayerNorm(embeds)
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class Kiwi(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embeds)
        self.position_embedding = nn.Embedding(block_size, embeds)
        self.blocks = nn.Sequential(*[Block(embeds, heads=heads) for _ in range(layers)])
        self.final_norm = nn.LayerNorm(embeds)
        self.head = nn.Linear(embeds, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape

        tok = self.token_embedding(index)
        pos = self.position_embedding(torch.arange(T, device=device))
        x = tok + pos
        x = self.blocks(x)
        x = self.final_norm(x)
        logits = self.head(x)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for iter in range(max_new_tokens):
            index_crop = index[:, -block_size:]
            logits, loss = self.forward(index_crop)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
            print(f"\r<{'*' * int(10 * iter/max_new_tokens)}{' ' * (10 - int(10*iter/max_new_tokens))}>", end='', flush=False)
        print("\r<**********>")
        return index

with open('kiwi.pkl', 'rb') as f:
    model = pickle.load(f)



C:\Users\jrbro\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))

In [6]:
%%time
ctx = torch.zeros((1, 1), dtype=torch.long, device=device)
generated = decode(model.generate(ctx, max_new_tokens=1000)[0].tolist())
print(generated)

<**********>



  Your of Knoxed’s jokes CockI of Rading’s Etology!”

So nothing like and “niasguable (that existady,”--“I pudding vitels, and so that.”

The Perfective passates or Luveramu’s I skeen as very lok ardo.

[Illustration]


SECTION III.
OF VERBS.

Nominative, governed little of those: asPhasis, and This, Disjunctive, Jilia,
Draēlitya&c.



COnjunction knews with a preson in takin which the gentmetian of skulenby. For it is not uts
than a nice other worth is and that every of the formed, which the
Comicalities muse, quite draw, and dravkabuting addressed afford a
personsion, or requirent know. Avering it to a celtbetive to _doe_ the heat--ewhich will the slowly into-morrious principle will you and sell
reasticate; and whom drunked and as last nent case in the affact,
  Wermarkins, but are some vulgar is mistles body excited of
   ERáVE MIST AND OF wIRD VESS INTENSS,

  FRONINubs TO BENGo,
   Lord R. Pentons of which To relatice, Victud, indev0 Scound,
    would say that danc

In [10]:
prompt = 'He is a '
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=500)[0].tolist())
print(generated_chars)

<**********>
He is a dessertative of them, which, in conduce, and green, number, “William a
spard,” &c.

SECTION SUBJECE.

1. Wo pronouns relate for prouns which we shall toolowitys as
shocking of certain and that alread supor pronounciations: at is a
“Dictor jock that gives;”
the Brothong make, that which can scorreover dispose. _Bin_, _any_, _an_, _an_, _a_, _an_, _en_:_, are,
and _are_ inspectably the addjective, and of the substantives; and althengs it is
also hing which in the definitive comparative shall never
